# Test optical flow for point tracking

In [21]:
import cv2
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
import math
import pandas as pd
from scipy.signal import savgol_filter
from scipy.signal import medfilt
import ast

import video

In [22]:
video_number = "2"
# Define the relative path to the video file
notebook_dir = Path().resolve()
project_root = notebook_dir.parent.parent
video_path = project_root / "data" / f"recording_{video_number}" / f"Recording_{video_number}.mp4" 
video_path = str(video_path)

# Load the video
cap = cv2.VideoCapture(video_path)

# Check
print(f"Opened: {cap.isOpened()}, FPS: {cap.get(cv2.CAP_PROP_FPS)}, Total Frames: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}")

Opened: True, FPS: 59.94005994005994, Total Frames: 276.0


Import points

In [23]:
# Define the path to the CSV file
input_data_path = project_root / "data" / "auxiliary_data" / "lane_points" / f"lane_points_processed_{video_number}.csv"

# Load the CSV file into a DataFrame
points_df = pd.read_csv(input_data_path)

Tracking of the top right point

In [24]:
# Reset the video to the beginning
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Read the first frame
ret, old_frame = cap.read()
if not ret:
    print("Cannot read video")
    cap.release()
    exit()

# Convert to grayscale
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Select the top-right point of the first frame
point = (int(points_df.iloc[0]['up_left_x']), int(points_df.iloc[0]['up_left_y']))

point_frame = old_frame.copy()
cv2.circle(point_frame, point, 5, (0, 0, 255), 2)
cv2.imshow("Frame", point_frame)

# create the array of the point to track
old_points = np.array([[point[0], point[1]]], dtype=np.float32).reshape(-1, 1, 2)

# store points
points = [point]

# Tracking loop
while True:
    ret, new_frame = cap.read()
    if not ret:
        break

    new_gray = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    new_points, status, error = cv2.calcOpticalFlowPyrLK(
        old_gray, new_gray, old_points, None
    )

    if status[0] == 1:
        x, y = new_points.ravel()

        # Update the previous frame and points
        old_gray = new_gray.copy()
        old_points = new_points.reshape(-1, 1, 2)
        points.append((int(x), int(y)))



Generate video

In [25]:
def add_point_to_frame(video_frame, point, color=(0, 255, 0), radius=5):
    """Add a point to the video frame."""
    cv2.circle(video_frame, point, radius, color, -1)
    return video_frame

In [26]:
# Reset the video to the beginning
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Define the codec and create a VideoWriter object to save the modified frames
output_path = project_root / "data" / f"recording_{video_number}"  / "Point_Tracking.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec for MP4 format
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(str(output_path), fourcc, fps, (frame_width, frame_height))


# Loop through each frame in the video
frame_index = 0
while frame_index < len(points):
    ret, video_frame = cap.read()
    if not ret:
        print("End of video or failed to read the frame at iteration", frame_index)
        break

    # draw the lines on the frame   
    modified_frame = add_point_to_frame(video_frame, points[frame_index], color=(0, 255, 0), radius=5)


    # Write the modified frame to the output video
    out.write(modified_frame)

    # Increment the frame index
    frame_index += 1

# Release the video capture and writer objects
# cap.release()
out.release()

print(f"Adjusted video saved to {output_path}")

Adjusted video saved to C:\Users\miche\OneDrive\Documenti\GitHub\bowling-analysis\data\recording_2\Point_Tracking.mp4
